## Bitcoin data set

In [45]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [48]:
data_bt_usd = pd.read_csv("csv files/BTC_USD.csv")
data_bt_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      1820 non-null   object
 1   Price     1820 non-null   object
 2   Open      1820 non-null   object
 3   High      1820 non-null   object
 4   Low       1820 non-null   object
 5   Vol.      1820 non-null   object
 6   Change %  1820 non-null   object
dtypes: object(7)
memory usage: 99.7+ KB


In [25]:
df = data_bt_usd.drop(['Open','High','Low','Vol.','Change %'], axis=1)

In [26]:
df = df.rename(columns={'Date': 'Tarih', 'Price': 'Şimdi'})
df

,Tarih,Şimdi
0,12/25/2022,"16,839.0"
1,12/24/2022,"16,864.0"
2,12/23/2022,"16,802.0"
3,12/22/2022,"16,838.0"
4,12/21/2022,"16,845.0"
...,...,...
1815,01/05/2018,"16,917.0"
1816,01/04/2018,"15,160.0"
1817,01/03/2018,"15,155.0"
1818,01/02/2018,"14,709.8"


In [27]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', ''))/10000)

## LSTM

In [28]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])




[1.6839 1.6864 1.6802 1.6838 1.6845 1.6925 1.6448] 1.6757
[1.6864 1.6802 1.6838 1.6845 1.6925 1.6448 1.6757] 1.6793
[1.6802 1.6838 1.6845 1.6925 1.6448 1.6757 1.6793] 1.6642
[1.6838 1.6845 1.6925 1.6448 1.6757 1.6793 1.6642] 1.7372
[1.6845 1.6925 1.6448 1.6757 1.6793 1.6642 1.7372] 1.781
[1.6925 1.6448 1.6757 1.6793 1.6642 1.7372 1.781 ] 1.7785
[1.6448 1.6757 1.6793 1.6642 1.7372 1.781  1.7785] 1.7228
[1.6757 1.6793 1.6642 1.7372 1.781  1.7785 1.7228] 1.7109
[1.6793 1.6642 1.7372 1.781  1.7785 1.7228 1.7109] 1.7132
[1.6642 1.7372 1.781  1.7785 1.7228 1.7109 1.7132] 1.7135
[1.7372 1.781  1.7785 1.7228 1.7109 1.7132 1.7135] 1.7246
[1.781  1.7785 1.7228 1.7109 1.7132 1.7135 1.7246] 1.6844
[1.7785 1.7228 1.7109 1.7132 1.7135 1.7246 1.6844] 1.7096
[1.7228 1.7109 1.7132 1.7135 1.7246 1.6844 1.7096] 1.6974
[1.7109 1.7132 1.7135 1.7246 1.6844 1.7096 1.6974] 1.7128
[1.7132 1.7135 1.7246 1.6844 1.7096 1.6974 1.7128] 1.689
[1.7135 1.7246 1.6844 1.7096 1.6974 1.7128 1.689 ] 1.7091
[1.7246 1.6844 1

In [29]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
57/57 [==============================] - 2s 8ms/step - loss: 0.7934
Epoch 2/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0491
Epoch 3/200
57/57 [==============================] - 1s 9ms/step - loss: 0.0395
Epoch 4/200
57/57 [==============================] - 0s 8ms/step - loss: 0.0349
Epoch 5/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0336
Epoch 6/200
57/57 [==============================] - 0s 7ms/step - loss: 0.0301
Epoch 7/200
57/57 [==============================] - 0s 8ms/step - loss: 0.0299
Epoch 8/200
57/57 [==============================] - 0s 8ms/step - loss: 0.0279
Epoch 9/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0274
Epoch 10/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0266
Epoch 11/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0251
Epoch 12/200
57/57 [==============================] - 0s 6ms/step - loss: 0.0248
Epoch 13/200
57/57 [=================

In [30]:
# demonstrate prediction
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], df['Şimdi'][6]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

## Daily 

In [31]:
print(f"Prediction: {round(yhat[0][0]*10000)} $ Bitcoin/USD") # onluk olarak yukarı yuvarlandı.

Prediction: 16945 $ Bitcoin/USD


## Weekly

In [32]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2],df['Şimdi'][3], df['Şimdi'][4], df['Şimdi'][5], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3], df['Şimdi'][4], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2] ,float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_5), float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_6 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_6), float(yhat_5),float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_7 = model.predict(x_input, verbose=0)

x_input = array([float(yhat_7), float(yhat_6), float(yhat_5),float(yhat_4), float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_8 = model.predict(x_input, verbose=0)


yhat   = float(yhat)*10000
yhat_2 = float(yhat_2)*10000
yhat_3 = float(yhat_3)*10000
yhat_4 = float(yhat_4)*10000
yhat_5 = float(yhat_5)*10000
yhat_6 = float(yhat_6)*10000
yhat_7 = float(yhat_7)*10000
yhat_8 = float(yhat_8)*10000


In [33]:
print(f"Prediction -> Pazartesi:{round(yhat,2)} Salı:{round(yhat_2,2)} Carsamba:{round(yhat_3,2)} Persembe:{round(yhat_4,2)} Cuma:{round(yhat_5,2)} Cumartesi:{round(yhat_6,2)} Pazar:{round(yhat_7,2)}  $ Euro/USD ")

Prediction -> Pazartesi:16945.0 Salı:17367.31 Carsamba:17443.58 Persembe:17459.49 Cuma:17453.56 Cumartesi:17443.63 Pazar:17443.62  $ Euro/USD 


In [34]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Week -> Prediction: {round(yhat_8,2)} $ Bitcoin/USD ")


Next Week -> Prediction: 17449.05 $ Bitcoin/USD 


## Monthly -> Burada bir sorun var. Garip bir hatası bulunmakta.

In [40]:
import array
exchange = array.array('f',[])

for i in range(len(df)):
    if i % 7 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])

In [41]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.6839 1.6757 1.7109 1.7128 1.6432 1.6281 1.6345] 2.0931999683380127
[1.6757 1.7109 1.7128 1.6432 1.6281 1.6345 2.0932] 2.06469988822937
[1.7109 1.7128 1.6432 1.6281 1.6345 2.0932 2.0647] 1.9585000276565552
[1.7128 1.6432 1.6281 1.6345 2.0932 2.0647 1.9585] 1.926900029182434
[1.6432 1.6281 1.6345 2.0932 2.0647 1.9585 1.9269] 1.9444400072097778
[1.6281  1.6345  2.0932  2.0647  1.9585  1.9269  1.94444] 1.906599998474121
[1.6345  2.0932  2.0647  1.9585  1.9269  1.94444 1.9066 ] 1.881700038909912
[2.0932  2.0647  1.9585  1.9269  1.94444 1.9066  1.8817 ] 1.9423999786376953
[2.0647  1.9585  1.9269  1.94444 1.9066  1.8817  1.9424 ] 2.1826000213623047
[1.9585  1.9269  1.94444 1.9066  1.8817  1.9424  2.1826 ] 2.000200033187866
[1.9269  1.94444 1.9066  1.8817  1.9424  2.1826  2.0002 ] 1.955780029296875
[1.94444 1.9066  1.8817  1.9424  2.1826  2.0002  1.95578] 2.1510000228881836
[1.9066  1.8817  1.9424  2.1826  2.0002  1.95578 2.151  ] 2.430000066757202
[1.8817  1.9424  2.1826  2.0002  1.95578 2

In [42]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
8/8 [==============================] - 1s 7ms/step - loss: 4.6765
Epoch 2/200
8/8 [==============================] - 0s 7ms/step - loss: 0.7997
Epoch 3/200
8/8 [==============================] - 0s 7ms/step - loss: 0.5459
Epoch 4/200
8/8 [==============================] - 0s 7ms/step - loss: 0.4206
Epoch 5/200
8/8 [==============================] - 0s 7ms/step - loss: 0.3243
Epoch 6/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2863
Epoch 7/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2753
Epoch 8/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2355
Epoch 9/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2312
Epoch 10/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2561
Epoch 11/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2402
Epoch 12/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2248
Epoch 13/200
8/8 [==============================] - 0s 7ms/st

In [43]:
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4],exchange[5],exchange[6]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)


In [44]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Month -> Prediction: {round(yhat[0][0]*10000)} $ Bitcoin/USD ")


Next Month -> Prediction: 16935 $ Bitcoin/USD 
